In [ ]:
import feedparser
import os
import numpy as np
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
import redis
from redis.commands.search.field import VectorField, TextField
from redis.commands.search.query import Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
import re
from redis.commands.search.field import TagField
from tenacity import retry
from tenacity import wait_random_exponential
from time import sleep
from tqdm import tqdm
import tiktoken as tk
import os
import spacy
import tiktoken as tk
from openai import OpenAI
import requests

import re
import spacy
os.environ["OPENAI_API_BOOK_KEY"]

In [2]:
redis_host = "localhost"
redis_port = "6379"
redis_password = ""

conn = redis.Redis(host=redis_host, 
                   port=redis_port, 
                   password=redis_password, 
                   encoding='utf-8', 
                   decode_responses=True)

p = conn.pipeline(transaction=False)

# TextProcessor
The TextProcessor class provides text processing utilities, including 
1. generating embeddings using OpenAI, 
2. token counting, 
3. splitting text into token-limited chunks while preserving sentence boundaries.

In [3]:
class TextProcessor:
    def __init__(self, client, model="en_core_web_sm", embedding_model="text-embedding-ada-002"):
        """Initialize the processor with OpenAI client and spaCy model."""
        self.client = client
        self.nlp = spacy.load(model)
        self.embedding_model = embedding_model

    def get_embedding(self, text):
        """Generate text embedding using OpenAI's text-embedding-ada-002 model."""
        response = self.client.embeddings.create(input=text, model=self.embedding_model)
        return response.data[0].embedding

    def count_tokens(self, text, encoding_name="cl100k_base"):
        """Count the number of tokens in a string using TikToken."""
        encoding = tk.get_encoding(encoding_name)
        return len(encoding.encode(text, disallowed_special=()))

    def split_text(self, text, max_tokens, overlap=0):
        """Split text into token-limited chunks while preserving sentence boundaries."""
        sentences = [sent.text for sent in self.nlp(text).sents]
        token_counts = [self.count_tokens(sent) for sent in sentences]

        chunks, start_idx = [], 0
        while start_idx < len(sentences):
            chunk, token_sum = [], 0
            for i in range(start_idx, len(sentences)):
                if token_sum + token_counts[i] > max_tokens:
                    break
                chunk.append(sentences[i])
                token_sum += token_counts[i]

            chunks.append(" ".join(chunk))
            start_idx += max(1, len(chunk) - overlap)

        return chunks

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_BOOK_KEY"))
text_processor = TextProcessor(openai_client)

In [4]:
resume = """
Zhe Sun, Data Scientist
Phone: 412-518-9494 Email: zhesun.work@gmail.com 
LinkedIn: https://www.linkedin.com/in/zhe-sun-datascientist/ GitHub: https://github.com/GeneSUN 
SUMMARY
Data Scientist with expertise in Fraud Detection, Causal Inference, Generative AI and Time-Series Forecasting, familiarity with big data processing frameworks such as Hadoop and Spark, delivering impactful solutions for Fortune 500 companies like Verizon and Walmart Labs.
EDUCATION
·	Ph.D. in Civil Engineering,                                                                          2023 University of Pittsburgh
·	Master in Structural Engineering,                                                                 2018 University of Pittsburgh
·	Bachelor's degree						        2016 China University of Petroleum
EXPERIENCE
Data Scientist, 	                            				       09/2023 - Present, Verizon Contract
·	Developed ETL pipelines and predictive models for churn prediction, and anomaly detection across Verizon's Wi-Fi and 5G services.
·	Delivered scalable solutions using Spark, Forecasting models, and ML/DL techniques.
·	Collaborated with cross-functional teams to analyze multi-terabyte datasets, enabling real-time insights and improved customer experience.
Data Scientist Intern, 		                                         05/2022 - 08/2022, Walmart Labs - Sunnyvale, CA
·	Conducted Causal Inference to evaluate the impact of self-checkout advertisements on revenue for 1,000+ Walmart stores and millions of customers.
·	Developed evaluation models using Diff-in-Diff, Propensity Score Matching, Meta-learners, GLM, and XGBoost, delivering actionable recommendations to marketing teams.
PROJECTS
Wi-Fi and 5G Home Internet Scoring System Development                              Verizon, 2024/01- Present
·	Designed scoring systems for Wi-Fi and 5G Home services, aggregating KPIs (10 for Wi-Fi, 7 for 5G) using statistical and domain expertise.
·	Built an Extender Recommendation Model achieving 82% accuracy, 78% precision, and 82% recall to optimize customer network performance.
·	Developed a 5G Home Churn prediction model using Time Series Classification (CNN, ResNets, KNN) methods to achieve 78% accuracy for proactive customer retention.
5G Network Station Anomaly Detection		                                               Verizon, 2024/06- 2024/09
·	Analyzed critical features of 5G stations during the transition from 4G to 5G, identifying physical patterns and essential variables. 
·	Developed a global anomaly detection model using probabilistic time series forecasting and deep learning techniques, with both univariate and multivariate approaches.
·	Enabled early detection of 5G station anomalies, reducing downtime and ensuring seamless service upgrades during the 4G to 5G transition.
Cellular Station (eNodeB) Anomaly Detection for Vendor Transition              Verizon, 2023/09- 2023/12
·	Developed an ETL to monitor eNodeB stations during transitions from Nokia to Samsung. 
·	Implemented a multi-level anomaly detection framework analyzing daily performance across 11 features for national eNodeB stations. 
Measurement of Store Self-Checkout Advertisement                                        Walmart, 2022/05- 2022/08  
·	Quantified the impact of self-checkout advertisements on revenue across 1000+ Walmart stores and millions of customers.
·	Developed evaluation models using Causal Inference algorithms (Diff-in-diff, Propensity-score model, Meta-learner, GLM, and XGBoost).
CERTIFICATIONS AND SKILLS
•	Certification of Amazon Web Services Certified Machine Learning – Specialty
•	Databricks Certified Associate Developer for Apache Spark 3.0 and Data Engineer Associate
•	Microsoft Certified: Power BI Data Analyst Associate 
•	DeepLearning.AI Certified: Natural Language Processing
TECHNICAL SKILLS:
•	Programming: Python, Scala, SQL
•	Big Data: Apache Spark, Databricks, Hadoop
•	Data Visualization: Power BI, Tableau
•	Machine Learning: Scikit-learn, TensorFlow, PyTorch, sktime
•	Cloud Platforms: AWS, GCP

"""
text_processor.count_tokens(resume)


880

In [8]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_BOOK_KEY"))

text_processor = TextProcessor(openai_client)

text = ("The FIFA Women's World Cup 2023 was an exciting event. "
        "Teams from all around the world participated. "
        "The final match was held in Sydney, Australia. "
        "The USA team performed exceptionally well but lost in the semifinals. "
        "Spain won the championship, marking their first-ever title in Women's World Cup history.")

print("🔹 Text Embedding:", text_processor.get_embedding("Machine learning is fascinating."))
print("🔹 Token Count:", text_processor.count_tokens(text))
print("🔹 Text Chunks:", text_processor.split_text(text, max_tokens=15, overlap=1))


🔹 Text Embedding: [-0.026537960395216942, 0.00856251735240221, 0.016943268477916718, -0.016722550615668297, 0.0003645480319391936, 0.015307365916669369, -0.00018257831106893718, 0.005878210533410311, -0.013333897106349468, -0.04063787683844566, 0.007679649628698826, 0.02908269688487053, -0.011217611841857433, -0.01203556265681982, -0.0011465918505564332, 0.011646062135696411, 0.036613039672374725, 0.01167202927172184, 0.0059723397716879845, -0.014022015035152435, -0.028978830203413963, 0.01935817115008831, -0.0029244988691061735, -0.032406434416770935, -0.020215071737766266, 0.003937200177460909, 0.004216341767460108, -0.03100423328578472, -0.0067967819049954414, -0.0077121080830693245, 0.02185097336769104, -0.005008325912058353, -0.015605983324348927, -0.03832684084773064, -0.008523567579686642, -0.012055037543177605, 0.0032068865839391947, -0.0016172381583601236, -0.01668360084295273, -0.001975091639906168, 0.02608354389667511, 0.02090318873524666, -0.002741109114140272, -0.018033869

This code 
1. fetches RSS feed entries, scrapes blog post details using BeautifulSoup, 
2. processes the text into chunks, 
3. generates embeddings with OpenAI,  
4. stores them in Redis.



# BlogPostScraper

In [5]:
class BlogPostScraper:
    def __init__(self, post_url, text_processor):
        self.post_url = post_url
        self.text_processor = text_processor
        self.soup = self._fetch_content()
        
        self.title = self._extract_title()
        self.description = self._extract_description()
        self.publish_date = self._extract_publish_date()
        self.body = self._extract_body()
        self.chunks = self._split_article()
    
    def _fetch_content(self):
        """Fetch the HTML content from the blog post URL."""
        response = requests.get(self.post_url)
        return BeautifulSoup(response.text, 'html.parser')
    
    def _extract_title(self):
        """Extract the blog post title."""
        try:
            title = self.soup.find('h1', {'class': 'post-title'}).text
            return title.replace("| Amit Bahree's (useless?) insight!", "")
        except AttributeError:
            return ""
    
    def _extract_description(self):
        """Extract the blog post description."""
        try:
            return self.soup.find('div', {'class': 'post-description'}).text
        except AttributeError:
            return ""
    
    def _extract_publish_date(self):
        """Extract the publish date of the blog post."""
        try:
            temp = self.soup.find('div', {'class': 'post-meta'}, {'span', 'title'}).text
            match = re.search(r"(\w+\s\d+,\s\d+)", temp)
            return match.group(1) if match else ""
        except AttributeError:
            return ""
    
    def _extract_body(self):
        """Extract the blog post body."""
        try:
            return self.soup.find('div', {'class': 'post-content'}).text
        except AttributeError:
            return ""
    
    def _split_article(self):
        """Split the article body into chunks."""
        return self.text_processor.split_text(self.body, max_tokens=3000, overlap=10) if self.body else []
    
    def get_article_data(self):
        """Return the extracted blog post data as a dictionary."""
        return {
            'title': self.title,
            'description': self.description,
            'publish_date': self.publish_date,
            'body': self.body,
            'chunks': self.chunks
        }


In [6]:
url = "https://blog.desigeek.com/index.xml"
feed = feedparser.parse(url)

blog_posts = len(feed.entries)
print("Number of blog posts: ", blog_posts)

max_posts = 10

import hashlib
for i, post in enumerate(feed.entries[:max_posts] ):
    # report progress
    print("Create embedding and save for entry #", i, " of ", blog_posts)
    scraper = BlogPostScraper(post.link, text_processor)
    article_data = scraper.get_article_data()
    print(f"Number of chunks: {len(article_data['chunks'])}")
    print( article_data.keys() )

    for j, chunk in enumerate(tqdm(article_data["chunks"])):
        vector = text_processor.get_embedding(chunk)
        vector = np.array(vector).astype(np.float32).tobytes() # Stores it as binary (.tobytes()) for efficient Redis storage.

        # Create a unique key using URL hash
        url_hash = hashlib.md5(post.link.encode()).hexdigest()
        chunk_key = f"post:{url_hash}_{j}"

        if not conn.exists(chunk_key):  # Check if key already exists
            post_hash = {
                "url": post.link,
                "title": article_data["title"],
                "description": article_data["description"],
                "publish_date": article_data["publish_date"],
                "content": chunk,
                "embedding": vector
            }

        conn.hset(name=f"post:{i}_{j}", mapping=post_hash)

p.execute()

print("Vector upload complete. 🤘")


Number of blog posts:  1311
Create embedding and save for entry # 0  of  1311
Number of chunks: 11
dict_keys(['title', 'description', 'publish_date', 'body', 'chunks'])


100%|██████████| 11/11 [00:05<00:00,  2.02it/s]


Create embedding and save for entry # 1  of  1311
Number of chunks: 13
dict_keys(['title', 'description', 'publish_date', 'body', 'chunks'])


100%|██████████| 13/13 [00:05<00:00,  2.40it/s]


Create embedding and save for entry # 2  of  1311
Number of chunks: 12
dict_keys(['title', 'description', 'publish_date', 'body', 'chunks'])


...

Create embedding and save for entry # 8  of  1311
Number of chunks: 11
dict_keys(['title', 'description', 'publish_date', 'body', 'chunks'])


100%|██████████| 11/11 [00:02<00:00,  4.92it/s]


Create embedding and save for entry # 9  of  1311
Number of chunks: 11
dict_keys(['title', 'description', 'publish_date', 'body', 'chunks'])


100%|██████████| 11/11 [00:02<00:00,  4.01it/s]

Vector upload complete. 🤘


In [7]:
url = "https://blog.desigeek.com/index.xml"
feed = feedparser.parse(url)
len(feed.entries)

post = feed.entries[0] 

scraper = BlogPostScraper(post.link, text_processor)
article_data = scraper.get_article_data()
print(f"Number of chunks: {len(article_data['chunks'])}")
print( article_data.keys() )

chunk =  article_data["chunks"][0]
vector = text_processor.get_embedding(chunk)
vector = np.array(vector).astype(np.float32).tobytes() # Stores it as binary (.tobytes()) for efficient Redis storage.

Number of chunks: 11
dict_keys(['title', 'description', 'publish_date', 'body', 'chunks'])


In [ ]:
chunk_vector=[]
for i in range( len(article_data["chunks"]) ):
    chunk =  article_data["chunks"][i]
    vector = text_processor.get_embedding(chunk)
    chunk_vector.append( vector )
embeddings_matrix = np.array(chunk_vector)

if the embeddings are small, we can use cosine_similarity to find the most related pair of query/chunk;

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query = "Neural networks are used in AI."

embedding_model="text-embedding-ada-002"
query_embedding = openai_client.embeddings.create(input=query, model=embedding_model).data[0].embedding

similarities = cosine_similarity([query_embedding], embeddings_matrix)[0]

most_similar_index = np.argmax(similarities)
most_similar_text = article_data["chunks"][most_similar_index]

print(f"Query: {query}")
print(f"Most Similar Text: {most_similar_text}")
print(f"Similarity Score: {similarities[most_similar_index]:.4f}")


Query: Neural networks are used in AI.
Most Similar Text: In the world of AI, there’s a thrill,With “Generative AI in Action,” you’ll skill. From prompts to fine-tuning,Your projects are blooming,Grab your copy, and master the drill!
Similarity Score: 0.8032


## upload chunks and embedding vector to vector base (redis)

connect to redis

In [34]:
redis_host = "localhost"
redis_port = "6379"
redis_password = ""

conn = redis.Redis(host=redis_host, 
                   port=redis_port, 
                   password=redis_password, 
                   encoding='utf-8', 
                   decode_responses=True)

p = conn.pipeline(transaction=False)

upload chunk to redis

In [ ]:
post_hash = {
    "url": post.link,
    "title": article_data["title"],
    "description": article_data["description"],
    "publish_date": article_data["publish_date"],
    "content": chunk,
    "embedding": vector
}
from pprint import pprint

pprint(post_hash)

conn.hset(name=f"post:{0}_{0}", mapping=post_hash)

"""
❓ Does the Second Script Auto-Index Documents?
🔹 No, Redis does not automatically index new documents after you create an index.
🔹 You must store documents using HSET, and Redis will index them automatically only if their key matches the defined prefix (post:).
"""

{'content': 'I hope you find it amusing!AI seeks to find,\n'
            'Humor in human design,\n'
            'Error, not aligned.',
 'description': 'AI-generated memes that outline their experience working with '
                'humans',
 'embedding': b'\xb6[\xd9\xbc\x95\xc6\x82\xbc\xb5I\xf3:a\xf8\xe4:\xc8\xd1R<'
              b'4\xae\xa2<\x15zJ\xbc,\xd2\xbb;\xba\x88\x98\xbc*\x1d\x83:'
              b'\xddb\xa59}\x08q:<\x8a\x89<3r\xdf\xbb\xc0I&<e\xd1i;\xddk\x18='
              b'VLl<:\xa2\x80<.\xc3\xb7\xbc(,\x87\xbcBT\n=\x84/\x1f\xbc'
              b'\xfb\x8d\x8a\xbc\x83&\xac\xbc\xcd\xb3\xca\xbbu\x80\xc4<'
              b'h\x1f\x13\xbd4Qu;\x8e\x96\xe1\xba\x88\x08\xa4<\xeb\xff\x99\xbb'
              b'\xb4\x9d\xad\xbc\x8a\xbd\xdc\xbc\xf9\xa5\x81\xbc\x18\xa7\t<'
              b'}e\x9e;\xd0\xd7\x16<\xa5\xee\xd2<\x02.\xae\xbcC\xd6r<\x19)\xf2;'
              b'\xb7dL\xba\x93\xab\xa9\xbba\xf8d\xbc\xb9U\xc8<U\xa9\x19<'
              b'*\x1d\x83:\x0b\xb6Z\xbc\xd5\x86>;5\x96\xab<}e\x1e;\xfd

0

create index

## Search
1. Full-Text Search (Keyword-Based) 
2. Vector Search (Semantic Similarity)


In [ ]:
import os
import numpy as np
from redis.commands.search.query import Query
import redis
from openai import OpenAI

class RedisSearch:
    def __init__(self, redis_client, openai_client, text_processor, max_token=4096, debug_message=False):
        self.redis_client = redis_client
        self.openai_client = openai_client
        self.text_processor = text_processor
        self.max_token = max_token
        self.debug_message = debug_message


    def text_search(self, query_text):
        """Perform a text-based search in Redis."""
        query = Query(query_text).return_fields("title", "url", "description")
        results = self.redis_client.ft("posts").search(query)
        
        return [(doc.title, doc.url) for doc in results.docs]

    def embed_query(self, query):
        """Convert the query into an embedding vector."""
        print("Generating embedding for query...")
        embedding = self.openai_client.embeddings.create(input=query, model="text-embedding-ada-002")
        query_vector = embedding.data[0].embedding
        return np.array(query_vector).astype(np.float32).tobytes()



    def hybrid_search(self, query, top_k=3, search_fields="*"):
        """Perform a hybrid vector search in Redis using an embedded query and return top-k results."""
        try:
            query_vector = self.embed_query(query)
            query_obj = Query(f"{search_fields}=>[KNN {top_k} @embedding $vector AS score]") \
                .return_fields("url", "title", "publish_date", "description", "content", "score") \
                .sort_by("score") \
                .dialect(2)

            results = self.redis_client.ft("posts").search(query_obj, query_params={"vector": query_vector})

            if results.total == 0:
                print("No matching results found.")
                return []
            elif results.total < top_k:
                print(f"Only {results.total} results found.")

            return results
        except Exception as e:
            print(f"❌ Redis search error: {e}")
            return []


    def retrieve_relevant_posts(self, query):
        """Retrieve relevant blog posts within a token budget."""
        search_results = self.hybrid_search(query)
        token_budget = self.max_token - self.text_processor.count_tokens(query) - 2000

        if self.debug_message:
            print(f"Token budget: {token_budget}")

        message = ('Use the blog post below to answer the subsequent question. ' \
                   'If the answer cannot be found in the articles, write ' \
                   '"Sorry, I could not find an answer in the blog posts."')
        
        question = f"\n\nQuestion: {query}"

        if search_results:
            for i, post in enumerate(search_results.docs):
                next_post = f'\n\nBlog post:\n"""\n{post.content}\n"""'
                new_token_usage = self.text_processor.count_tokens(message + question + next_post)
                if new_token_usage < token_budget:
                    if self.debug_message:
                        print(f"Token usage: {new_token_usage}")
                    message += next_post
                else:
                    break
        else:
            print("No results found")

        return message + question

    def ask_gpt(self, query : str) -> str:
        message = self.retrieve_relevant_posts( query)
        
        if self.debug_message:
            print(message)
        
        # Ask GPT
        messages = [ 
            {"role": "system", 
            "content": "You answer questions in summary from the blog posts."},
            {"role": "user",
                "content": message},]
        
        if self.debug_message:
            print("Length of messages: ", len(messages))
            if self.debug_message:
                print("Total tokens: ", self.text_processor.count_tokens(messages))
                print(messages)

        response = self.openai_client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.5,
            max_tokens=2000,
            top_p=0.95)
        
        response_message = response.choices[0].message.content
        
        return response_message
    
searcher = RedisSearch(conn, openai_client, text_processor)
searcher.ask_gpt("generative AI advancements")

Generating embedding for query...


'The blog posts discuss the advancements in Generative AI, particularly in content creation. It mentions that AI has the potential to revolutionize content creation by generating natural-sounding audio summaries from text. This is considered a game-changer for authors, creators, and educators. As AI continues to advance, more opportunities to improve how we access information are anticipated. The posts also highlight the importance of responsible AI practices such as prompt engineering, ethical considerations, and security measures when implementing these technologies.'

**bb text search with exact word **

In [ ]:
search_results = searcher.text_search("ai")
for title, url in search_results:
    print(f"Title: {title}, URL: {url}")


Connected to Redis
Title: 🎉Announcing My New Book: Generative AI in Action📚, URL: https://blog.desigeek.com/post/2024/10/book-release-genai-in-action/
Title: AI generated Podcast for my book: Generative AI in Action 🎧, URL: https://blog.desigeek.com/post/2024/10/ai-generated-book-podcast/
Title: 📚 My new book "Generative AI in Action", URL: https://blog.desigeek.com/post/2023/11/announcing-gen-ai-book/
Title: AI working with humans, URL: https://blog.desigeek.com/post/2023/07/ai-working-with-humans/
Title: Shedding Light on the Art of Prompt Engineering, URL: https://blog.desigeek.com/post/2024/03/shedding-light-on-the-art-of-prompt-engineering/
Title: An introduction to Mixture of Experts (MoE), URL: https://blog.desigeek.com/post/2025/01/intro-to-mixture-of-experts/


In [68]:
query = "Latest AI advancements"
search_results = searcher.hybrid_search("Latest AI advancements")

if search_results.total > 0:
    print("\n🔹 Search Results 🔹\n")
    for i, doc in enumerate(search_results.docs):
        print(f"📌 **Result {i+1}:**")
        print(f"🆔 ID: {doc.id}")
        print(f"🔗 URL: {doc.url}")
        print(f"📖 Title: {doc.title}")
        print(f"🗓️ Publish Date: {doc.publish_date}")
        print(f"📝 Description: {doc.description}")
        print(f"📊 Similarity Score: {doc.score}")
        print(f"📄 Content (Preview): {doc.content[:500]}...")  # Show only first 500 characters
        print("-" * 80)
else:
    print("⚠️ No results found.")


Generating embedding for query...

🔹 Search Results 🔹

📌 **Result 1:**
🆔 ID: post:3_5
🔗 URL: https://blog.desigeek.com/post/2024/10/ai-generated-book-podcast/
📖 Title: AI generated Podcast for my book: Generative AI in Action 🎧
🗓️ Publish Date: October 13, 2024
📝 Description: 
📊 Similarity Score: 0.160163700581
📄 Content (Preview): The author emphasizes the need for careful evaluation, monitoring, and scalability when deploying Generative AI models in a production environment. Your browser does not support the audio element. Book Podcast - using single sourceConclusionAI-generated podcasts showcase the potential of AI to revolutionize content creation. Generating natural-sounding audio summaries from text is a game-changer for authors, creators, and educators. As AI continues to advance, we anticipate more opportunities to...
--------------------------------------------------------------------------------
📌 **Result 2:**
🆔 ID: post:0_3
🔗 URL: https://blog.desigeek.com/post/2024/10/book

In [51]:
max_token = 4096 
debug_message=False
query = "Latest AI advancements"
token_budget = max_token - text_processor.count_tokens(query) - 2000
if debug_message:
    print(f"Token budget: {token_budget}")

message = 'Use the blog post below to answer the subsequent question. \
        If the answer cannot be found in the articles, write \
        "Sorry, I could not find an answer in the blog posts."'

question = f"\n\nQuestion: {query}"

if search_results:
    for i, post in enumerate(search_results.docs):
        next_post = f'\n\nBlog post:\n"""\n{post.content}\n"""'
        new_token_usage = text_processor.count_tokens(message + question + next_post)
        if new_token_usage < token_budget:
            if debug_message:
                print(f"Token usage: {new_token_usage}")
            message += next_post
        else:
            break
else:
    print("No results found")


In [45]:
def view_index(conn, index_name="posts"):
    try:
        info = conn.execute_command('FT.INFO', index_name)
        for i in range(0, len(info), 2):
            print(f"{info[i]}: {info[i+1]}")
    except Exception as e:
        print(f"⚠️ Failed to retrieve index details: {e}")

view_index(conn)

index_name: posts
index_options: []
index_definition: ['key_type', 'HASH', 'prefixes', ['post:'], 'default_score', '1']
attributes: [['identifier', 'url', 'attribute', 'url', 'type', 'TAG', 'SEPARATOR', ','], ['identifier', 'title', 'attribute', 'title', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'description', 'attribute', 'description', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'publish_date', 'attribute', 'publish_date', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'content', 'attribute', 'content', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'embedding', 'attribute', 'embedding', 'type', 'VECTOR', 'algorithm', 'HNSW', 'data_type', 'FLOAT32', 'dim', 1536, 'distance_metric', 'COSINE', 'M', 16, 'ef_construction', 200]]
num_docs: 112
max_doc_id: 112
num_terms: 4662
num_records: 22172
inverted_sz_mb: 0.6013574600219727
vector_index_sz_mb: 6.235511779785156
total_inverted_index_blocks: 4672
offset_vectors_sz_mb: 0.05369281768798828
doc_table_size_mb: 0.007922172546386719
so

In [25]:
doc_id = "post:0_0"  # Example document key
embedding = conn.hget(doc_id, "embedding")  # Retrieve stored binary vector

if embedding:
    vector_array = np.frombuffer(embedding, dtype=np.float32)  # Convert back to NumPy array
    print("Embedding vector:", vector_array)
else:
    print("No embedding found for the given document.")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb6 in position 0: invalid start byte

In [26]:
index_info = conn.execute_command("FT.INFO posts")  # "posts" is your index name

for i in range(0, len(index_info), 2):
    print(f"{index_info[i]}: {index_info[i+1]}")


index_name: posts
index_options: []
index_definition: ['key_type', 'HASH', 'prefixes', ['post:'], 'default_score', '1']
attributes: [['identifier', 'url', 'attribute', 'url', 'type', 'TAG', 'SEPARATOR', ','], ['identifier', 'title', 'attribute', 'title', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'description', 'attribute', 'description', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'publish_date', 'attribute', 'publish_date', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'content', 'attribute', 'content', 'type', 'TEXT', 'WEIGHT', '1'], ['identifier', 'embedding', 'attribute', 'embedding', 'type', 'VECTOR', 'algorithm', 'HNSW', 'data_type', 'FLOAT32', 'dim', 1536, 'distance_metric', 'COSINE', 'M', 16, 'ef_construction', 200]]
num_docs: 112
max_doc_id: 114
num_terms: 4662
num_records: 22679
inverted_sz_mb: 0.6051063537597656
vector_index_sz_mb: 6.2359771728515625
total_inverted_index_blocks: 5363
offset_vectors_sz_mb: 0.054886817932128906
doc_table_size_mb: 0.007922172546386719


In [29]:
import redis
import numpy as np

# Connect to Redis (with binary-safe settings)
conn = redis.Redis(
    host=redis_host,
    port=redis_port,
    password=redis_password,
    encoding='utf-8', 
    decode_responses=False  # Disable auto-decoding for binary data
)

# Specify the document key (e.g., "post:5_2" for blog post 5, chunk 2)
doc_key = "post:5_2"

# Retrieve the hash (all fields of the document)
doc_data = conn.hgetall(doc_key)

if not doc_data:
    print(f"Document {doc_key} not found.")
else:
    # Retrieve the embedding field
    embedding_binary = doc_data.get(b"embedding")  # Use bytes key for binary data

    if embedding_binary:
        # Convert binary data back to NumPy array
        embedding_vector = np.frombuffer(embedding_binary, dtype=np.float32)
        print(f"Embedding vector for {doc_key}:")
        print(embedding_vector)
    else:
        print(f"No embedding found for {doc_key}.")


Embedding vector for post:5_2:
[-0.0150971   0.01764171 -0.01155027 ...  0.01656131 -0.00984439
 -0.06601781]


In [32]:
embedding_vector = conn.hget("post:0_0", "embedding")
import numpy as np

embedding_vector = np.frombuffer(conn.hget("post:0_0", "embedding"), dtype=np.float32)
print(embedding_vector.shape)  # Should be (1536,)
embedding_vector

(1536,)


array([-0.02653299, -0.01596383,  0.00185614, ..., -0.00609309,
        0.00759658, -0.04301978], dtype=float32)

## How to Retrieve the Index of Embedding Vectors
Redis does not store the index as an explicit structure that you can query directly. Instead, the vector index is an internal data structure that enables fast nearest-neighbor searches.

However, you can check if a specific document is indexed by running: